In [10]:
hdfs_endpoint='hdfs://10.0.1.76:8020'
hdfs_path='/user/dominospark/small_data'
cnt=100
%env HDFS_PATH /user/dominospark/small_data
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python

#%env PATH /usr/lib/hadoop/bin:/bin:/usr/lib/spark/bin::/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/tmp:/opt/oracle/instantclient_12_1:/home/ubuntu/.local/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/share/scala/bin:/opt/oracle/instantclient_12_1:/home/ubuntu/.local/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/hadoop-hdfs/bin:/opt/hadoop/bin
!hdfs dfs -rm -r $HDFS_PATH
!echo $PYTHONPATH

env: HDFS_PATH=/user/dominospark/small_data
env: HADOOP_HOME=/usr/lib/hadoop
env: HADOOP_CONF_DIR=/etc/hadoop/conf
env: HADOOP_YARN_HOME=/usr/lib/hadoop
env: HADOOP_MAPRED_HOME=/usr/lib/hadoop
env: HADOOP_HDFS_HOME=/usr/lib/hadoop-hdfs
env: SPARK_HOME=/usr/lib/spark
env: SPARK_CONF_DIR=/etc/spark/conf
env: PYSPARK_PYTHON=/opt/conda/bin/python
env: PYSPARK_DRIVER_PYTHON=/opt/conda/bin/python
Deleted /user/dominospark/small_data
:/opt/spark/python/lib/py4j-0.10.7-src.zip:/opt/spark/python/


In [11]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random

In [12]:
print(hdfs_endpoint)
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", hdfs_endpoint) \
.getOrCreate()


hdfs://10.0.1.76:8020


In [13]:
data = []

for i in range(cnt):
    data.append((i,random.randint(1,100),random.randint(1,100),random.randint(1,100)))

columns = StructType([ StructField("id", IntegerType(), True),
                       StructField("v1", IntegerType(), True),
                       StructField("v2", IntegerType(), True),
                       StructField("v3", IntegerType(), True) ])


df = sparkSession.createDataFrame(data=data, schema=columns)
df.write.csv(hdfs_path)


In [17]:
df_load = sparkSession.read.csv(hdfs_path,columns)
df_load.where(df_load.id < 5).show()


+---+---+---+---+
| id| v1| v2| v3|
+---+---+---+---+
|  0|  4| 23| 30|
|  1| 28| 46|  8|
|  2| 77| 15|  2|
|  3| 25| 47| 94|
|  4|  7|  4| 75|
+---+---+---+---+



In [6]:
sparkSession.stop()

In [7]:
!hdfs dfs -ls /user/dominospark/small_data

Found 2 items
-rw-r--r--   1 dominospark dominospark          0 2021-08-13 15:30 /user/dominospark/small_data/_SUCCESS
-rw-r--r--   1 dominospark dominospark       1165 2021-08-13 15:30 /user/dominospark/small_data/part-00000-782dc7bf-b733-4302-bc02-6d7efc2587ed-c000.csv
